# <span style="color: #b58bf7;"> Creating the Redux Store </span>

## <span style="color: #b58bf7;"> Intro </span>

The Redux store is created using the ``configureStore`` function from Redux Toolkit. ``configureStore`` requires that we pass in a ``reducer`` argument.

In [ ]:
import { configureStore } from '@reduxjs/toolkit'
import counterReducer from '../features/counter/counterSlice'

/**
 * configureStore({}) wraps createStore 
 * to provide simplified configuration options 
 * and good defaults. It can automatically combine 
 * your slice reducers, adds whatever 
 * Redux middleware you supply, 
 * includes redux-thunk by default, 
 * and enables use of the 
 * Redux DevTools Extension.
 */
export default configureStore({
  /**
   * interface ConfigureStoreOptions<
   *  S = any , 
   *  A extends Action = AnyAction  , 
   *  M extends Middlewares<S> = Middlewares<S> , 
   *  E extends Enhancers = Enhancers
   * >{
   *  reducer: Reducer<S, A> | ReducersMapObject<S, A>,
   *  middleware?: ((getDefaultMiddleware: CurriedGetDefaultMiddleware<S>) => M) | M;
   *  devTools?: boolean | DevToolsOptions,
   *  preloadedState?: PreloadedState<CombinedState<NoInfer<S>>>,
   *  enhancers?: E | ConfigureEnhancersCallback<E>
   * }
   */
  reducer: {
    domainFeature: sliceFeature
  },

  devTools: true,
})

Our application might be made up of many different features, and each of those features might have its own reducer function (object factory function `() => {return {}} `). When we call ``configureStore``, we can pass in all of the different reducers in an object. The key names in the object will define the keys in our final state value.

> We have a file named ``features/counter/counterSlice.js`` that exports a reducer function for the counter logic. We can import that ``counterReducer`` function here, and include it when we create the store.

>   When we pass in an object like ``{counter: counterReducer}``, that says that we want to have a ``state.counter`` section of our Redux state object, and that we want the ``counterReducer`` function to be in charge of deciding if and how to update the ``state.counter`` section whenever an action is dispatched.

Redux allows store setup to be customized with different kinds of plugins ("*middleware*" and "*enhancers*"). ``configureStore`` automatically adds several middleware to the store setup by default to provide a good developer experience, and also sets up the store so that the Redux DevTools Extension can inspect its contents.

## <span style="color: #b58bf7;"> Reducers and State Structure </span>

A **Redux store** **needs** to have a single "**root reducer**" function passed in when it's created. 

If we have **many different slice reducer** functions, **how do we get a single root reducer instead**, and how does this define the contents of the Redux store state?

### <span style="color: #b58bf7;"> ``combineReducers`` </span>

Redux has a function called ``combineReducers`` that does this for us automatically. It accepts an object full of slice reducers as its argument, and returns a function that calls each slice reducer whenever an action is dispatched. The result from each slice reducer are all combined together into a single object as the final result. We can do the same thing as the previous example using ``combineReducers``:

In [ ]:
const rootReducer = combineReducers({
    users: usersReducer,
    posts: postsReducer,
    comments: commentsReducer
})

## <span style="color: #b58bf7;"> Redux Slices </span>

A "slice" is a **collection of Redux reducer logic and actions for a single feature in your app**, typically defined together in a single file. The name comes from **splitting up the root Redux state object into multiple "slices" of state**.

In [ ]:
import { configureStore } from '@reduxjs/toolkit'
import usersReducer from '../features/users/usersSlice'
import postsReducer from '../features/posts/postsSlice'
import commentsReducer from '../features/comments/commentsSlice'

export default configureStore({
  reducer: {
    users: usersReducer,
    posts: postsReducer,
    comments: commentsReducer
  }
})

## <span style="color: #b58bf7;"> Creating Slice Reducers and Actions </span>

> Since we know that the ``counterReducer`` function is coming from ``features/counter/counterSlice.js``

In [ ]:
import { createSlice } from '@reduxjs/toolkit'

export const counterSlice = createSlice({
  name: 'counter',
  initialState: {
    value: 0
  },
  reducers: {
    increment: state => {
      // Redux Toolkit allows us to write "mutating" logic in reducers. It
      // doesn't actually mutate the state because it uses the immer library,
      // which detects changes to a "draft state" and produces a brand new
      // immutable state based off those changes
      state.value += 1
    },
    decrement: state => {
      state.value -= 1
    },
    incrementByAmount: (state, action) => {
      state.value += action.payload
    }
  }
})

export const { increment, decrement, incrementByAmount } = counterSlice.actions

export default counterSlice.reducer

> Redux Toolkit has a function called ``createSlice``, which takes care of the work of generating action type strings, action creator functions, and action objects. All you have to do is define a name for this slice, write an object that has some reducer functions in it, and it generates the corresponding action code automatically. The string from the name option is used as the first part of each action type, and the key ``name`` of each reducer function is used as the second part. So, the "``counter``" name + the "``increment``" reducer function generated an action type of ``{type: "counter/increment"}``. (After all, why write this by hand if the computer can do it for us!)

> In addition to the name field, createSlice needs us to pass in the initial state value for the reducers, so that there is a state the first time it gets called. In this case, we're providing an object with a value field that starts off at 0.

> createSlice automatically generates action creators with the same names as the reducer functions we wrote. We can check that by calling one of them and seeing what it returns:

In [ ]:
console.log(counterSlice.actions.increment())
// {type: "counter/increment"}

> It also generates the slice reducer function that knows how to respond to all these action types:



In [ ]:
const newState = counterSlice.reducer(
  { value: 10 },
  counterSlice.actions.increment()
)
console.log(newState)
// {value: 11}

## <span style="color: #b58bf7;"> Writing Async Logic with Thunks </span>